In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, f1_score

In [4]:
PATH = '/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/HC_h_fusion/hc_fused_readtext.npz'

data = np.load(PATH, allow_pickle=False)
X, y, ids = data["X"], data["y"], data["ids"]

# split by patient if multiple recordings per patient (derive patient key from id string)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# SVM (RBF) – needs scaling
svm = make_pipeline(StandardScaler(with_mean=True, with_std=True),
                    SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced"))
svm.fit(Xtr, ytr)
yp = svm.predict(Xte)
print("SVM  bal-acc:", balanced_accuracy_score(yte, yp), "macro-F1:", f1_score(yte, yp, average="macro"))


SVM  bal-acc: 1.0 macro-F1: 1.0


In [5]:
# RandomForest – scaling not necessary
rf = RandomForestClassifier(n_estimators=400, class_weight="balanced", max_depth=None, random_state=42)
rf.fit(Xtr, ytr)
yp = rf.predict(Xte)
print("RF   bal-acc:", balanced_accuracy_score(yte, yp), "macro-F1:", f1_score(yte, yp, average="macro"))

RF   bal-acc: 1.0 macro-F1: 1.0


# to check the credibility of the evaluation result


In [7]:
import re, numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, f1_score

# load fused
data = np.load(PATH, allow_pickle=False)
X, y, ids = data["X"], data["y"], data["ids"]

# patient id (e.g., "ID00_hc_..." -> "ID00")
patients = np.array([re.split(r'[_]', i)[0] for i in ids])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
svm = make_pipeline(StandardScaler(with_mean=True), SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced"))

ba, f1 = [], []
for tr, te in cv.split(X, y, groups=patients):
    svm.fit(X[tr], y[tr])
    yp = svm.predict(X[te])
    ba.append(balanced_accuracy_score(y[te], yp))
    f1.append(f1_score(y[te], yp, average="macro"))
print("GroupCV bal-acc mean±sd:", np.mean(ba), "±", np.std(ba))
print("GroupCV macro-F1 mean±sd:", np.mean(f1), "±", np.std(f1))


GroupCV bal-acc mean±sd: 1.0 ± 0.0
GroupCV macro-F1 mean±sd: 1.0 ± 0.0


In [8]:
rng = np.random.default_rng(0)
y_shuf = rng.permutation(y)
svm = make_pipeline(StandardScaler(), SVC(kernel="rbf", class_weight="balanced"))
svm.fit(X, y_shuf)
print("Permutation bal-acc (should be ~0.5):",
      balanced_accuracy_score(y_shuf, svm.predict(X)))


Permutation bal-acc (should be ~0.5): 0.7485119047619048


In [9]:
from sklearn.metrics import confusion_matrix
# after one CV fold:
print(confusion_matrix(y[te], yp))
print("Test size:", len(te))


[[4 0]
 [0 4]]
Test size: 8


# External Validation


I want to train the entire readText  and will use the entire spontaneous fusion for test

In [11]:
Train_PATH = '/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/HC_h_fusion/hc_fused_readtext.npz'

train_data = np.load(Train_PATH, allow_pickle=False)
X, y, ids = train_data["X"], train_data["y"], train_data["ids"]

Test_path = "/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/spontaneous_h_fusion/spontaneous_fused_readtext.npz"
test_data = np.load(Test_path, allow_pickle=False)
Xtest, ytest, idstest = test_data["X"], test_data["y"], test_data["ids"]

# SVM (RBF) – needs scaling
svm = make_pipeline(StandardScaler(with_mean=True, with_std=True),
                    SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced"))
svm.fit(X, y)
yp = svm.predict(Xtest)
print("SVM  bal-acc:", balanced_accuracy_score(ytest, yp), "macro-F1:", f1_score(ytest, yp, average="macro"))


SVM  bal-acc: 0.5 macro-F1: 0.3076923076923077


In [12]:
# RandomForest – scaling not necessary
rf = RandomForestClassifier(n_estimators=400, class_weight="balanced", max_depth=None, random_state=42)
rf.fit(X, y)
yp = rf.predict(Xtest)
print("RF   bal-acc:", balanced_accuracy_score(ytest, yp), "macro-F1:", f1_score(ytest, yp, average="macro"))

RF   bal-acc: 0.55 macro-F1: 0.4109090909090909


# Ablation

In [13]:
import numpy as np, re
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, f1_score

# Load the two fused files you created
src = np.load("/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/HC_h_fusion/hc_fused_readtext.npz", allow_pickle=False)
tgt = np.load("/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/spontaneous_h_fusion/spontaneous_fused_readtext.npz", allow_pickle=False)

X_src, y_src = src["X"], src["y"]
X_tgt, y_tgt = tgt["X"], tgt["y"]
D_TEXT, D_AUDIO, D_CLIP = int(src["D_text"]), int(src["D_audio"]), int(src["D_graph"])

sl_text  = slice(0, D_TEXT)
sl_audio = slice(D_TEXT, D_TEXT + D_AUDIO)
sl_clip  = slice(D_TEXT + D_AUDIO, D_TEXT + D_AUDIO + D_CLIP)

svm = make_pipeline(StandardScaler(with_mean=True, with_std=True),
                    SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced"))

def eval_one(slices):
    Xs = np.concatenate([X_src[:, s] for s in slices], axis=1)
    Xt = np.concatenate([X_tgt[:, s] for s in slices], axis=1)
    model = svm.fit(Xs, y_src)
    yp = model.predict(Xt)
    return (balanced_accuracy_score(y_tgt, yp),
            f1_score(y_tgt, yp, average="macro"))

for name, sls in {
    "text_only":[sl_text], "audio_only":[sl_audio], "clip_only":[sl_clip],
    "text+audio":[sl_text, sl_audio], "text+clip":[sl_text, sl_clip],
    "audio+clip":[sl_audio, sl_clip], "all_three":[sl_text, sl_audio, sl_clip],
}.items():
    ba, f1 = eval_one(sls)
    print(f"{name:11s} | BA {ba:.3f} | F1 {f1:.3f}")


text_only   | BA 0.500 | F1 0.308
audio_only  | BA 0.500 | F1 0.308
clip_only   | BA 0.500 | F1 0.308
text+audio  | BA 0.500 | F1 0.308
text+clip   | BA 0.500 | F1 0.308
audio+clip  | BA 0.500 | F1 0.308
all_three   | BA 0.500 | F1 0.308
